In [1]:
# VIAF LoC ISNI reconciliation à a partir d'une liste de noms
#script original https://github.com/ehanson8/linked-data-reconciliation-python-scripts
# -*- encoding: utf-8 -*- ajouté pour éviter l'erreur fréquente charset
# votre people.csv en entrée DOIT ETRE ENCODE EN UTF-8 première ligne name sans "", un nom par ligne, pas de ligne vide 
import requests
import csv
from fuzzywuzzy import fuzz
import json 
import urllib
import html

baseURL = 'http://viaf.org/viaf/search/viaf?query=local.personalNames+%3D+%22'
f = csv.writer(open('viafPeopleResults.csv', 'w', encoding='utf-8'))
f.writerow(['search'] + ['result'] + ['viaf'] + ['lc'] + ['isni'] + ['ratio']
           + ['partialRatio'] + ['tokenSort'] + ['tokenSet'] + ['avg'])
with open('people.csv', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        name = str(row['name'])
        rowEdited = urllib.parse.quote(name.strip())
        urlSuffix = '%22+and+local.sources+%3D+%22lc%22&sortKeys=holdingscount'
        urlSuffix2 = '&maximumRecords=1&httpAccept=application/rdf+json'
        url = baseURL + rowEdited + urlSuffix + urlSuffix2
        print(url)
        response = requests.get(url).content.decode('utf-8')
        try:
            startString = '<recordData xsi:type="ns1:stringOrXmlFragment">'
            startIndex = response.index(startString)
            endString = '</recordData>'
            endIndex = response.index(endString)
            print(endIndex, startIndex)
            response = response[startIndex + 47: endIndex]
            response = response.replace('&quot;', '"')
            response = json.loads(response)
            label = response['mainHeadings']['data']
            if isinstance(label, list):
                label = label[0]['text']
            else:
                label = label['text']
            label = html.unescape(label)
            print(label)
            viafid = response['viafID']
        except ValueError:
            label = ''
            viafid = ''
#calcul du ratio de similarité entre la chaine de caractères entrée et celles obtenues.             
        ratio = fuzz.ratio(name, label)
        partialRatio = fuzz.partial_ratio(name, label)
        tokenSort = fuzz.token_sort_ratio(name, label)
        tokenSet = fuzz.token_set_ratio(name, label)
        avg = (ratio + partialRatio + tokenSort + tokenSet) / 4

        if viafid != '':
            links = json.loads(requests.get('http://viaf.org/viaf/' + viafid
                               + '/justlinks.json').text)
            viafid = 'http://viaf.org/viaf/' + viafid
# tente le mapping avec Library Of Congress         
            try:
                lc = 'http://id.loc.gov/authorities/names/'
                lc = lc + json.dumps(links['LC'][0]).replace('"', '')
            except ValueError:
                lc = ''
# tente le mapping avec ISNI commenté ici car pas utile/redondant avec Viaf pour mon projet. Pour un résumé de la distinction entre les 2 http://moodle.abes.fr/pluginfile.php/5331/mod_resource/content/0/2015_S13_IdRef_%20Usages_pertinence_et_fonctions_de_VIAF_et_ISNI.pdf
#            try:
#                isni = 'http://isni.org/isni/'
#                isni = isni + json.dumps(links['ISNI'][0]).replace('"', '')
#            except ValueError:
#                isni = ''
        else:
            lc = ''
        f.writerow([name.strip()] + [label] + [viafid] + [lc]
                   + [ratio] + [partialRatio] + [tokenSort]
                   + [tokenSet] + [avg])
        
# si le script se déroule bien la sortie est un fichier viafPeopleResults.csv avec les colonnes search, result (attention aux homonymes) l'id Viaf l'id LoC et un ratio de similarité terme recherché/terme obtenu 

http://viaf.org/viaf/search/viaf?query=local.personalNames+%3D+%22Fussler%20Herman%20H.%22+and+local.sources+%3D+%22lc%22&sortKeys=holdingscount&maximumRecords=1&httpAccept=application/rdf+json
108492 820
Fussler, Herman Howe, 1914-....
http://viaf.org/viaf/search/viaf?query=local.personalNames+%3D+%22Hitler%20Adolf%22+and+local.sources+%3D+%22lc%22&sortKeys=holdingscount&maximumRecords=1&httpAccept=application/rdf+json
825334 822
Hitler, Adolf, 1889-1945.
http://viaf.org/viaf/search/viaf?query=local.personalNames+%3D+%22Mussolini%20Benito%22+and+local.sources+%3D+%22lc%22&sortKeys=holdingscount&maximumRecords=1&httpAccept=application/rdf+json
825333 821
Hitler, Adolf, 1889-1945.
http://viaf.org/viaf/search/viaf?query=local.personalNames+%3D+%22Blum%20L%C3%A9on%22+and+local.sources+%3D+%22lc%22&sortKeys=holdingscount&maximumRecords=1&httpAccept=application/rdf+json
364508 820
Blum, Léon, 1872-1950
http://viaf.org/viaf/search/viaf?query=local.personalNames+%3D+%22Lebrun%20Albert%22+and+

KeyError: 'ISNI'